# 实验说明

## 作业说明

### 目标：

训练一个玩数独的神经网络，并得到较高的准确率

### 背景：

数独是一种经典的逻辑游戏，玩家需要根据9×9盘面上的已知数字，推理出所有剩余空格的数字，并满足每一行、每一列、每一个粗线宫（3x3）内的数字均含数字1～9，不重复。本教程展示如何训练一个用于产生数独游戏解的神经网络模型，并评估其准确率和计算时间。该模型得到9x9每个位置上填入数字1～9的概率分布（共有9x9x9个概率）。

**训练过程**：将模型输出的概率分布与 ground truth 得到交叉熵 loss，并反向传播更新梯度。

**推理过程**：我们**逐个地**填入数字，过程为先选中某个位置，再选择1～9的一个数字进行填入。
循环执行以下操作，直到全部填充完毕：

1. 将 1x9x9 的数独游戏输入模型，输出 81x9 的概率分布

2. 每个位置如果需要填入，一定会选择数字1～9中概率最大的填入。即：

$num_{x,y} = argmax(P_{x,y,1},P_{x,y,2},\cdots,P_{x,y,9})$

3. 每次需要选择一个空位置进行填入，我们选择所有空位置 $(x,y)$ 中 $P_{x,y}$ 最大的填入。

$P_{x,y} = max(P_{x,y,1},P_{x,y,2},\cdots,P_{x,y,9}) $

$chosenP = argmax(P_{x,y})$

### 任务：

#### Q1： 训练模型

运行baseline，训练和评估模型，并记录平均损失。你可以使用自己的数独游戏，输出结果，以判断模型是否可以得到合理的结果。

由于数独可能存在多个合理解，因此只比较和 ground truth 的差别，并非是好的评估方式。请修改模型评估的代码，判断解是否合理，并计算模型得到合理解的概率。**请于q1.diff提交你的代码。**

#### Q2： 尝试数据增广

数独游戏可以尝试多种数据增广：左右翻转，上下翻转，旋转后性质不变；将1～9数字重新排列（shuffle）后，性质也不变。使用更小的样本（例如仅使用1000个训练数据），并修改代码，尝试使用更多的增广方式进行训练，得到Q1中的准确率尽可能高的模型。**请于q2.diff提交你的代码。**

#### Q3： 训练一个量化神经网络

从 baseline 模型中，提取量化神经网络。可以参考本文以了解量化的工作原理：[DoReFa-Net: Training Low Bitwidth Convolutional Neural Networks with Low Bitwidth Gradients](https://arxiv.org/abs/1606.06160)。

建议分别将量化权重和特征的位数设置为2和1，你也可以使用其他配置。你可以使用 ground truth 进行训练，也可以采用知识蒸馏的方式，尝试获得更好的效果。详见：[知识蒸馏教程](https://studio.brainpp.com/project/4719)。

完成以上任务，得到Q1中的准确率尽可能高的量化模型。 **请于q3.diff提交你的代码。**

## 数据集

来源：[1 million Sudoku games](https://www.kaggle.com/bryanpark/sudoku)，该数独数据集包含100万对9x9网格下的数独游戏以及解，存储于`./dataset/dataset-2115/sudoku.csv`。

## 文件存储
实验中生成的文件可以存储于 workspace 目录中。 查看工作区文件，该目录下的变更将会持久保存。 您的所有项目将共享一个存储空间，请及时清理不必要的文件，避免加载过慢。

## 实验步骤

1.导入库

In [1]:
import numpy as np
import megengine as mge
import megengine.module as M
import megengine.functional as F
import megengine.data.transform as T
import pandas as pd
import copy
from tqdm import tqdm
from typing import Tuple
from megengine import tensor
from sklearn.model_selection import train_test_split
from megengine.data import DataLoader, RandomSampler
from megengine.data.dataset import Dataset
from megengine.tensor import Parameter
from megengine.optimizer import Adam
from megengine.autodiff import GradManager, Function

2.设计模型结构，并实例化模型

In [2]:
# class Net(M.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv0 = M.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
#         self.relu0 = M.ReLU()
#         self.bn0 = M.BatchNorm2d(64)
#         self.conv1 = M.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
#         self.relu1 = M.ReLU()
#         self.bn1 = M.BatchNorm2d(64)
#         self.conv2 = M.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
#         self.relu2 = M.ReLU()
#         self.bn2 = M.BatchNorm2d(64)
#         self.conv3 = M.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
#         self.relu3 = M.ReLU()
#         self.bn3 = M.BatchNorm2d(64)
#         self.conv4 = M.Conv2d(64, 128, kernel_size=1, stride=1, padding=0)
#         self.relu4 = M.ReLU()
#         self.fc = M.Linear(10368, 81*9)

#     def forward(self, x):
#         x = self.conv0(x)
#         x = self.relu0(x)
#         x = self.bn0(x)
#         x = self.conv1(x)
#         x = self.relu1(x)
#         x = self.bn1(x)
#         x = self.conv2(x)
#         x = self.relu2(x)
#         x = self.bn2(x)
#         x = self.conv3(x)
#         x = self.relu3(x)
#         x = self.bn3(x)
#         x = self.conv4(x)
#         x = self.relu4(x)
#         x = F.flatten(x, 1)
#         x = self.fc(x)
#         x = F.reshape(x, (-1, 81, 9))
#         return x

# model = Net()

In [3]:
def uniform_quantize(k):
    class qfn(Function):
        def forward(self, input):
            if k == 32:
                out = input
            elif k == 1:
                out = F.sign(input)
            else:
                n = float(2 ** k - 1)
                out = F.round(input * n) / n
            return out

        def backward(self, grad_output):
            grad_input = F.copy(grad_output)
            return grad_input

    return qfn()


class weight_quantize_fn(M.Module):
    def __init__(self, w_bit):
        super(weight_quantize_fn, self).__init__()
        assert w_bit <= 8 or w_bit == 32
        self.w_bit = w_bit
        self.uniform_q = uniform_quantize(k=w_bit)

    def forward(self, x):
        if self.w_bit == 32:
            weight_q = x
        elif self.w_bit == 1:
            E = F.mean(F.abs(x)).detach()
            weight_q = self.uniform_q(x / E) * E
        else:
            weight = F.tanh(x)
            max_w = F.max(F.abs(weight)).detach()
            weight = weight / 2 / max_w + 0.5
            weight_q = max_w * (2 * self.uniform_q(weight) - 1)
        return weight_q


class activation_quantize_fn(M.Module):
    def __init__(self, a_bit):
        super(activation_quantize_fn, self).__init__()
        assert a_bit <= 8 or a_bit == 32
        self.a_bit = a_bit
        self.uniform_q = uniform_quantize(k=a_bit)

    def forward(self, x):
        if self.a_bit == 32:
            activation_q = x
        else:
            activation_q = self.uniform_q(F.clip(x, 0, 1))
            # print(np.unique(activation_q.detach().numpy()))
        return activation_q


def conv2d_Q_fn(w_bit):
    class Conv2d_Q(M.Conv2d):
        def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                     padding=0, dilation=1, groups=1, bias=True):
            super(Conv2d_Q, self).__init__(in_channels, out_channels, kernel_size, stride,
                                     padding, dilation, groups, bias)
            self.w_bit = w_bit
            self.quantize_fn = weight_quantize_fn(w_bit=w_bit)

        def forward(self, input, order=None):
            weight_q = self.quantize_fn(self.weight)
            # print(np.unique(weight_q.detach().numpy()))
            return F.conv2d(input, weight_q, self.bias, self.stride,
                        self.padding, self.dilation, self.groups)

    return Conv2d_Q


def linear_Q_fn(w_bit):
    class Linear_Q(M.Linear):
        def __init__(self, in_features, out_features, bias=True):
            super(Linear_Q, self).__init__(in_features, out_features, bias)
            self.w_bit = w_bit
            self.quantize_fn = weight_quantize_fn(w_bit=w_bit)

        def forward(self, input):
            weight_q = self.quantize_fn(self.weight)
            # print(np.unique(weight_q.detach().numpy()))
            return F.linear(input, weight_q, self.bias)

    return Linear_Q


In [4]:
wbit = 1
abit = 2

In [5]:
class Net(M.Module):
    def __init__(self):
        super(Net, self).__init__()
        Conv2d = conv2d_Q_fn(w_bit=wbit)
        Linear = linear_Q_fn(w_bit=wbit)
        self.conv0 = M.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        self.relu0 = M.ReLU()
        self.bn0 = M.BatchNorm2d(64)
        
        self.conv1 = Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu1 = M.ReLU()
        self.bn1 = M.BatchNorm2d(64)
        self.a1 = activation_quantize_fn(a_bit=abit)
        
        self.conv2 = Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = M.ReLU()
        self.bn2 = M.BatchNorm2d(64)
        self.a2 = activation_quantize_fn(a_bit=abit)
        
        self.conv3 = Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = M.ReLU()
        self.bn3 = M.BatchNorm2d(64)
        self.a3 = activation_quantize_fn(a_bit=abit)
        
        self.conv4 = Conv2d(64, 128, kernel_size=1, stride=1, padding=0)
        self.relu4 = M.ReLU()
        self.a4 = activation_quantize_fn(a_bit=abit)
        
        self.fc = M.Linear(10368, 81*9)

    def forward(self, x):
        x = self.conv0(x)
        x = self.relu0(x)
        x = self.bn0(x)
        
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.bn1(x)
        x = self.a1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.bn2(x)
        x = self.a2(x)
        
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.bn3(x)
        x = self.a3(x)
        
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.a4(x)
        
        x = F.flatten(x, 1)
        x = self.fc(x)
        x = F.reshape(x, (-1, 81, 9))
        return x

model = Net()

3.准备训练数据，设置优化器

In [6]:
def get_data(file): 
    data = pd.read_csv(file)
    feat_raw = data['quizzes']
    label_raw = data['solutions']
    feat = []
    label = []

    for i in feat_raw:
        x = np.array([int(j) for j in i]).reshape((1,9,9))
        feat.append(x)
    feat = np.array(feat)
    feat = feat/9
    feat -= .5 
    for i in label_raw:
        x = np.array([int(j) for j in i]).reshape((1,81)) - 1
        label.append(x)   
    label = np.array(label)
    
    del(feat_raw)
    del(label_raw)    

    x_train, x_test, y_train, y_test = train_test_split(feat, label, test_size=0.2, random_state=42)
    
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = get_data('./dataset/dataset-2115/sudoku.csv')

class TrainDataset(Dataset):
    def __init__(self,x_train,y_train):
        super().__init__()
        self.data_x = x_train
        self.data_y = y_train
    def __getitem__(self, index:int) -> Tuple:
        return self.data_x[index], self.data_y[index]
    def __len__(self) -> int:
        return len(self.data_x)

train_dataset = TrainDataset(x_train, y_train)
train_sampler = RandomSampler(dataset = train_dataset, batch_size=32)
train_dataloader = DataLoader(dataset = train_dataset, sampler=train_sampler)

opt = Adam(model.parameters(), lr = 1e-3)

4.训练模型

In [7]:
epochs = 2
for epoch in range(epochs):
    iter = 0
    with tqdm(total=train_dataloader.__len__() / 100, desc="epoch {}:training process".format(epoch)) as tq:
        for i, (feat, label) in enumerate(train_dataloader):
            
            gm = GradManager().attach(model.parameters())
            with gm:
                logits = model(tensor(feat))
                label = label.reshape(32, 81) 
                loss = F.loss.cross_entropy(logits, label, axis=2)
                
                iter += 1
                if iter % 100 == 0:
                    tq.set_postfix({"loss": "{0:1.5f}".format(loss.numpy().item()),})
                    tq.update(1)
                    
                gm.backward(loss)
            opt.step()
            opt.clear_grad()

# mge.save(model, "1.mge")
# print("saved")

epoch 0:training process: 100%|██████████| 250/250.0 [04:12<00:00,  1.01s/it, loss=0.44785]
epoch 1:training process: 100%|██████████| 250/250.0 [04:10<00:00,  1.00s/it, loss=0.38324]
epoch 2:training process: 100%|██████████| 250/250.0 [04:12<00:00,  1.01s/it, loss=0.39672]
epoch 3:training process: 100%|██████████| 250/250.0 [04:13<00:00,  1.01s/it, loss=0.38967]
epoch 4:training process: 100%|██████████| 250/250.0 [04:11<00:00,  1.00s/it, loss=0.36270]


AttributeError: Can't pickle local object 'conv2d_Q_fn.<locals>.Conv2d_Q'

5.模型评估

你需要修改这部分的代码，以完成作业Q1。

In [8]:
# model = Net()
# model = mge.load("1.mge")
model.eval()

eval_dataset = TrainDataset(x_test, y_test)
eval_sampler = RandomSampler(dataset = eval_dataset, batch_size=32)
eval_dataloader = DataLoader(dataset = eval_dataset, sampler=eval_sampler)

average_loss = 0
iter = 0
with tqdm(total=eval_dataloader.__len__() / 100, desc="evaluating process") as tq:
    for i, (feat, label) in enumerate(eval_dataloader):
        
            logits = model(tensor(feat))
            label = label.reshape(32, 81)
            loss = F.loss.cross_entropy(logits, label, axis=2)
            
            iter += 1
            if iter % 100 == 0:
                tq.set_postfix({"loss": "{0:1.5f}".format(loss.numpy().item()),})
                tq.update(1)
            average_loss += loss.numpy().item()

print(average_loss / eval_dataloader.__len__())

evaluating process:  99%|█████████▉| 62/62.5 [00:26<00:00,  2.34it/s, loss=0.37625]

0.3761109496879578


6.输入一个你自己的数独游戏，并查看模型的输出结果。

In [9]:
# model = Net()
# model = mge.load("1.mge")
model.eval()

def norm(x):
    return (x/9)-.5

def denorm(x):
    return (x+.5)*9

def inference_sudoku(sample):
    feat = copy.copy(sample)
    
    while(1):
        out = F.softmax(model(tensor(feat.reshape((1, 1, 9, 9)))), axis= 2)
        out = out.reshape(81, 9)

        pred = F.argmax(out, axis=1).reshape((9, 9))+1 
        prob = np.around(F.max(out, axis=1).reshape((9, 9)), 2) 
        feat = denorm(feat).reshape((9, 9))
        mask = ( feat == 0 )
        if mask.sum() == 0:
            break
            
        prob_new = prob * mask 
        ind = F.argmax(prob_new) 
        x, y = (ind // 9), (ind % 9)
        
        val = pred[x][y].numpy()
        feat[x][y] = int(val)
        feat = norm(feat)
    
    return feat

def solve_sudoku(game):
    
    game = game.replace('\n', '')
    game = game.replace(' ', '')
    game = np.array([int(j) for j in game]).reshape((9,9,1))
    game = norm(game)
    game = inference_sudoku(game)
    return game

game = '''
          4 0 9 5 3 2 0 0 0
          7 0 3 0 8 0 0 9 0
          5 0 0 0 0 7 0 3 0
          0 5 0 0 0 1 9 7 0
          6 0 0 7 0 9 0 0 8
          0 4 7 2 0 0 0 5 0
          0 2 0 6 0 0 0 0 9
          8 0 0 0 9 0 3 0 5
          3 0 0 8 2 0 0 1 0
      '''

game = solve_sudoku(game)

print('solved puzzle:\n')
print(game.astype("uint8"))

solved puzzle:

[[4 8 9 5 3 2 7 6 1]
 [7 1 3 4 8 6 5 9 2]
 [5 6 2 9 1 7 8 3 4]
 [2 5 8 3 4 1 9 7 6]
 [6 3 1 7 5 9 2 4 8]
 [9 4 7 2 6 8 1 5 3]
 [1 2 5 6 7 3 4 8 9]
 [8 7 6 1 9 4 3 2 5]
 [3 9 4 8 2 5 6 1 7]]


## 结果展示

展示为baseline效果。

![sudoku.png](https://data.megengine.org.cn/megstudio/images/sudoku.png)